# Whiskey clustering 

This example examines 86 Scotch whiskies that have been rated on
a five-point scale for 12 flavor characteristics: Body, sweetness, smoky, medicinal, tobacco, honey, spicy, winey, nutty, malty, fruity, and floral.
This data set comes from a book on the classification of Scotch whisky based on
flavors by David Wishart (2002) and can be found [online](https://www.niss.org/sites/default/files/ScotchWhisky01.txt).

We'll use [Tablesaw](https://tablesaw.tech/) to store and manipulate our data
and the clustering and projection classes from the [Smile](http://haifengl.github.io/)
machine learning library. So, we'll add those libraries to the classpath and define some imports to simplify access to the classes we need.

In [120]:
%%classpath add mvn
tech.tablesaw tablesaw-beakerx 0.36.0
com.github.haifengl smile-core 1.5.3

In [121]:
%import static tech.tablesaw.aggregate.AggregateFunctions.*
%import tech.tablesaw.api.*
%import tech.tablesaw.columns.*
%import smile.clustering.*
%import smile.projection.PCA

We'll also enable a BeakerX display widget for Tablesaw tables.

In [122]:
tech.tablesaw.beakerx.TablesawDisplayer.register()
OutputCell.HIDDEN

### Exploring the data

We start by loading the data and printing its shape and structure.

In [123]:
records = Table.read().csv("../resources/whiskey.csv")
records.shape()

86 rows X 14 cols

In [124]:
records.structure()

### K-means clustering

K-means is the most common form of “centroid” clustering. Unlike classification, clustering is an unsupervised learning method. The categories are not predetermined. Instead, the goal is to search for natural groupings in the dataset, such that the members of each group are similar to each other and different from the members of the other groups. The K represents the number of groups to find.

In [125]:
cols = ["Body", "Sweetness", "Smoky", "Medicinal", "Tobacco", "Honey",
        "Spicy", "Winey", "Nutty", "Malty", "Fruity", "Floral"]
data = records.as().doubleMatrix(*cols)
kMeans = new KMeans(data, 5)

K-Means distortion: 404.48848
Clusters of 86 data points of dimension 12:
  0	   23 (26.7%)
  1	    7 ( 8.1%)
  2	   31 (36.0%)
  3	    2 ( 2.3%)
  4	   23 (26.7%)


In [126]:
Table clusters = Table.create("Clusters", records.stringColumn("Distillery"),
                              DoubleColumn.create("Cluster", kMeans.getClusterLabel()))
clusters = clusters.sortAscendingOn("Cluster", "Distillery")

We have some nice classifications but how do we visualize them? A common approach would be to reduce our twelve "dimensions" (our features) onto a smaller number of dimensions, typically 2 or 3.

In [ ]:
pca = new PCA(data)
dimensions = 2
numClusters = 5
pca.projection = dimensions
projected = pca.project(data)
clusterer = new KMeans(data, numClusters)
labels = clusterer.clusterLabel.collect { "Cluster " + (it + 1) }
records = records.addColumns(
    *(0..<dimensions).collect { idx ->
        DoubleColumn.create("PCA${idx+1}", (0..<data.size()).collect{
            projected[it][idx]
        })},
    StringColumn.create("Cluster", labels),
    DoubleColumn.create("Centroid", [10] * labels.size())
)

In [139]:
colors = [
    'Cluster 1': new Color(255, 31, 31),
    'Cluster 2': new Color(15, 255, 15),
    'Cluster 3': new Color(0, 0, 222),
    'Cluster 4': new Color(255, 255, 32),
    'Cluster 5': new Color(155, 3, 155)
]
title = "Clusters x Principal Components w/ centroids"
plot = new Plot(title: title, xLabel: 'PCA1', yLabel: 'PCA2')
plot << new Points(x: records.column('PCA1').toList(),
                   y: records.column('PCA2').toList(),
                   color: records.column('Cluster').toList().collect{ colors[it] })